## Pytorch 빠른 튜토리얼

In [ ]:
import torch
from torch.utils.data import DataLoader #데이터셋을 순회가능(iterable)한 객체로 변경
from torchvision import datasets #샘플과 정답(label)을 저장
from torch import nn
from torchvision.transforms import ToTensor

## Step 1. Load Torch Models

PyTorch는 TorchText, TorchVision, TorchAudio와 같은 도메인별 라이브러리를 제공하며, 이들 모두 데이터셋을 포함하고 있습니다. 이 튜토리얼에서는 TorchVision 데이터셋을 사용할 것입니다.

torchvision 모듈은 CIFAR, COCO 등과 같은 실제 세계의 비전 데이터를 위한 Dataset 객체들을 포함하고 있습니다. 이 튜토리얼에서는 FashionMNIST 데이터셋을 사용합니다. 모든 TorchVision Dataset은 샘플과 레이블을 각각 수정하기 위한 transform 및 target_transform 두 가지 인자를 포함합니다.

In [ ]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [ ]:
training_data

In [ ]:
test_data

데이터셋을 DataLoader에 인자로 전달합니다. 이를 통해 데이터셋에 대한 반복 가능한 객체가 생성되며, 자동 배치, 샘플링, 셔플링 및 멀티프로세스 데이터 로딩이 지원됩니다. 여기서는 배치 크기를 64로 정의하는데, 이는 DataLoader 반복 가능 객체가 64개의 특성과 레이블을 가진 배치를 반환할 것임을 의미합니다.

In [ ]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

## Step 2. Create Torch Models

PyTorch에서 신경망을 정의하기 위해, nn.Module을 상속받는 클래스를 만듭니다. 네트워크의 층은 __init__ 함수 내에서 정의되며, 데이터가 네트워크를 통과하는 방식은 forward 함수에서 명시합니다. 신경망의 연산을 가속화하기 위해, 가능한 경우 GPU나 MPS(메탈 성능 셰이더)로 네트워크를 옮깁니다.

In [ ]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

In [ ]:
# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device) #GPU로 신경망을 이동해 연산을 가속
print(model)

## Step 3. Training Torch Models

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [ ]:
size = len(train_dataloader.dataset)

In [ ]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train() #모델을 학습모드로,
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device) #샘플과 정답을 GPU로 전달

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad() #optimizer의 gradient 초기화
        loss.backward() #loss
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

## Step 4. Saving Models

In [ ]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

## Step 5. Loading Models

In [ ]:
model = NeuralNetwork()
model.load_state_dict(torch.load("model.pth"))

In [ ]:
model

In [ ]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

In [ ]:
model(next(iter(test_dataloader))[0]).shape

In [ ]:
pred = model(x)

In [ ]:
x.shape

# What's Next in Learning More About PyTorch?

- [자신의 커스텀 데이터셋을 위한 파이토치 데이터셋 클래스 정의하기](https://pytorch.org/tutorials/beginner/basics/data_tutorial.html)
- [파이토치 공식 튜토리얼 투어하기](https://pytorch.org/tutorials/)